In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_excel(r'C:\Users\Lenovo\Desktop\ML_health\pbicr_11\Item_Scores_anx.xlsx')
df.head()

,Number,liveStrict,hukou,homeNum,chaiqian,home_mianji,zhaiwu,family_income,gender,age,...,Stress1,Stress2,Stress3,Emontion_regulate1,Emontion_regulate2,Emontion_regulate3,Emontion_regulate4,Emontion_regulate5,Emontion_regulate6,risk_anxiety
0,1,1,1,2,0,4,1,8,2,2,...,2,2,3,1,1,3,0,3,3,0
1,2,1,1,1,0,3,0,1,2,2,...,2,5,3,3,3,3,3,3,1,0
2,3,1,1,3,1,4,1,2,2,5,...,2,2,2,1,2,2,0,0,3,0
3,4,2,2,1,0,4,0,3,2,1,...,1,1,1,0,1,4,3,3,3,0
4,5,1,1,0,0,1,0,1,1,2,...,5,4,4,4,3,2,3,3,3,1


In [4]:
X = df.drop('risk_anxiety', axis = 1)
X.shape

(11031, 97)

In [5]:
y=df['risk_anxiety']
y.shape

(11031,)

In [6]:
pd.Series(y).value_counts()

risk_anxiety
0    9534
1    1497
Name: count, dtype: int64

In [7]:
smote = SMOTE()

In [8]:
X.dtypes

Number                int64
liveStrict            int64
hukou                 int64
homeNum               int64
chaiqian              int64
                      ...  
Emontion_regulate2    int64
Emontion_regulate3    int64
Emontion_regulate4    int64
Emontion_regulate5    int64
Emontion_regulate6    int64
Length: 97, dtype: object

In [9]:
X = X.apply(pd.to_numeric, errors='coerce')

In [10]:
X.dtypes

Number                int64
liveStrict            int64
hukou                 int64
homeNum               int64
chaiqian              int64
                      ...  
Emontion_regulate2    int64
Emontion_regulate3    int64
Emontion_regulate4    int64
Emontion_regulate5    int64
Emontion_regulate6    int64
Length: 97, dtype: object

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42) 

In [12]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [13]:
print(y_train.value_counts())  # 原始训练集的类别分布
print(y_train_resampled.value_counts())  # 过采样后的类别分布

risk_anxiety
0    8591
1    1336
Name: count, dtype: int64
risk_anxiety
0    8591
1    8591
Name: count, dtype: int64


In [14]:
df2=pd.concat([y_train_resampled, X_train_resampled], axis=1)

In [15]:
df2.to_excel(r'C:\Users\Lenovo\Desktop\ML_health\pbicr_11\Item_Scores_anx_smote.xlsx',index = False)

In [3]:
df3 = pd.read_excel(r'C:\Users\Lenovo\Desktop\ML_health\pbicr_11\Item_Scores_anx_smote.xlsx')

In [4]:
X = df3.drop('risk_anxiety', axis = 1)
X.shape

(17182, 97)

In [5]:
y=df3['risk_anxiety']
y.shape

(17182,)

In [6]:
X = df3.drop(['risk_anxiety', 'Number'], axis=1)

In [7]:
X.shape

(17182, 96)

In [8]:
y=df3['risk_anxiety']
y.shape

(17182,)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42) 

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 创建决策树分类器
model = DecisionTreeClassifier()

# 创建 KFold 交叉验证分割器
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 初始化指标列表
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []

# 进行10×10交叉验证
for train_index, test_index in kf.split(X_train):
    # 分割数据，使用 .iloc 来根据行索引选择数据
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # 训练模型
    model.fit(X_train_fold, y_train_fold)
    
    # 进行预测
    y_pred_fold = model.predict(X_test_fold)
    
    # 计算并存储每个指标
    accuracy = accuracy_score(y_test_fold, y_pred_fold)
    accuracy_scores.append(accuracy)
    
    precision = precision_score(y_test_fold, y_pred_fold, average='macro')
    precision_scores.append(precision)
    
    recall = recall_score(y_test_fold, y_pred_fold, average='macro')
    recall_scores.append(recall)
    
    f1 = f1_score(y_test_fold, y_pred_fold, average='macro')
    f1_scores.append(f1)
    
    # 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
    if len(set(y_test_fold)) == 2:
        y_scores_fold = model.predict_proba(X_test_fold)[:, 1]  # 获取正类的概率
        roc_auc = roc_auc_score(y_test_fold, y_scores_fold)
        roc_auc_scores.append(roc_auc)
    
    # 计算Specificity
    conf_mat_fold = confusion_matrix(y_test_fold, y_pred_fold)
    tn, fp, fn, tp = conf_mat_fold.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificity_scores.append(specificity)

# 输出每个指标的平均值
print("Average Accuracy:", sum(accuracy_scores) / len(accuracy_scores))
print("Average Precision:", sum(precision_scores) / len(precision_scores))
print("Average Recall:", sum(recall_scores) / len(recall_scores))
print("Average F1 Score:", sum(f1_scores) / len(f1_scores))
if roc_auc_scores:
    print("Average ROC AUC:", sum(roc_auc_scores) / len(roc_auc_scores))
print("Average Specificity:", sum(specificity_scores) / len(specificity_scores))

Average Accuracy: 0.8689770126380735
Average Precision: 0.8696183782483764
Average Recall: 0.8689020402434954
Average F1 Score: 0.8688733317791293
Average ROC AUC: 0.8689020402434956
Average Specificity: 0.8490516092803884


In [24]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


model = GaussianNB()

# 创建 KFold 交叉验证分割器
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 初始化指标列表
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []

# 进行10×10交叉验证
for train_index, test_index in kf.split(X_train):
    # 分割数据，使用 .iloc 来根据行索引选择数据
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # 训练模型
    model.fit(X_train_fold, y_train_fold)
    
    # 进行预测
    y_pred_fold = model.predict(X_test_fold)
    
    # 计算并存储每个指标
    accuracy = accuracy_score(y_test_fold, y_pred_fold)
    accuracy_scores.append(accuracy)
    
    precision = precision_score(y_test_fold, y_pred_fold, average='macro')
    precision_scores.append(precision)
    
    recall = recall_score(y_test_fold, y_pred_fold, average='macro')
    recall_scores.append(recall)
    
    f1 = f1_score(y_test_fold, y_pred_fold, average='macro')
    f1_scores.append(f1)
    
    # 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
    if len(set(y_test_fold)) == 2:
        y_scores_fold = model.predict_proba(X_test_fold)[:, 1]  # 获取正类的概率
        roc_auc = roc_auc_score(y_test_fold, y_scores_fold)
        roc_auc_scores.append(roc_auc)
    
    # 计算Specificity
    conf_mat_fold = confusion_matrix(y_test_fold, y_pred_fold)
    tn, fp, fn, tp = conf_mat_fold.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificity_scores.append(specificity)

# 输出每个指标的平均值
print("Average Accuracy:", sum(accuracy_scores) / len(accuracy_scores))
print("Average Precision:", sum(precision_scores) / len(precision_scores))
print("Average Recall:", sum(recall_scores) / len(recall_scores))
print("Average F1 Score:", sum(f1_scores) / len(f1_scores))
if roc_auc_scores:
    print("Average ROC AUC:", sum(roc_auc_scores) / len(roc_auc_scores))
print("Average Specificity:", sum(specificity_scores) / len(specificity_scores))

Average Accuracy: 0.8323742234479621
Average Precision: 0.8330482740166092
Average Recall: 0.8323458652321192
Average F1 Score: 0.8322486617620312
Average ROC AUC: 0.9055264411900594
Average Specificity: 0.8093482446020058


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


model = LogisticRegression(max_iter=2000)

# 创建 KFold 交叉验证分割器
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 初始化指标列表
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []

# 进行10×10交叉验证
for train_index, test_index in kf.split(X_train):
    # 分割数据，使用 .iloc 来根据行索引选择数据
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # 训练模型
    model.fit(X_train_fold, y_train_fold)
    
    # 进行预测
    y_pred_fold = model.predict(X_test_fold)
    
    # 计算并存储每个指标
    accuracy = accuracy_score(y_test_fold, y_pred_fold)
    accuracy_scores.append(accuracy)
    
    precision = precision_score(y_test_fold, y_pred_fold, average='macro')
    precision_scores.append(precision)
    
    recall = recall_score(y_test_fold, y_pred_fold, average='macro')
    recall_scores.append(recall)
    
    f1 = f1_score(y_test_fold, y_pred_fold, average='macro')
    f1_scores.append(f1)
    
    # 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
    if len(set(y_test_fold)) == 2:
        y_scores_fold = model.predict_proba(X_test_fold)[:, 1]  # 获取正类的概率
        roc_auc = roc_auc_score(y_test_fold, y_scores_fold)
        roc_auc_scores.append(roc_auc)
    
    # 计算Specificity
    conf_mat_fold = confusion_matrix(y_test_fold, y_pred_fold)
    tn, fp, fn, tp = conf_mat_fold.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificity_scores.append(specificity)

# 输出每个指标的平均值
print("Average Accuracy:", sum(accuracy_scores) / len(accuracy_scores))
print("Average Precision:", sum(precision_scores) / len(precision_scores))
print("Average Recall:", sum(recall_scores) / len(recall_scores))
print("Average F1 Score:", sum(f1_scores) / len(f1_scores))
if roc_auc_scores:
    print("Average ROC AUC:", sum(roc_auc_scores) / len(roc_auc_scores))
print("Average Specificity:", sum(specificity_scores) / len(specificity_scores))

Average Accuracy: 0.8959450791959733
Average Precision: 0.8959931199276161
Average Recall: 0.8959307056964683
Average F1 Score: 0.8959177793648768
Average ROC AUC: 0.9596006445754496
Average Specificity: 0.8945831111556297


In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

model = KNeighborsClassifier()

# 创建 KFold 交叉验证分割器
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 初始化指标列表
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []

# 进行10×10交叉验证
for train_index, test_index in kf.split(X_train):
    # 分割数据，使用 .iloc 来根据行索引选择数据
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # 训练模型
    model.fit(X_train_fold, y_train_fold)
    
    # 进行预测
    y_pred_fold = model.predict(X_test_fold)
    
    # 计算并存储每个指标
    accuracy = accuracy_score(y_test_fold, y_pred_fold)
    accuracy_scores.append(accuracy)
    
    precision = precision_score(y_test_fold, y_pred_fold, average='macro')
    precision_scores.append(precision)
    
    recall = recall_score(y_test_fold, y_pred_fold, average='macro')
    recall_scores.append(recall)
    
    f1 = f1_score(y_test_fold, y_pred_fold, average='macro')
    f1_scores.append(f1)
    
    # 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
    if len(set(y_test_fold)) == 2:
        y_scores_fold = model.predict_proba(X_test_fold)[:, 1]  # 获取正类的概率
        roc_auc = roc_auc_score(y_test_fold, y_scores_fold)
        roc_auc_scores.append(roc_auc)
    
    # 计算Specificity
    conf_mat_fold = confusion_matrix(y_test_fold, y_pred_fold)
    tn, fp, fn, tp = conf_mat_fold.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificity_scores.append(specificity)

# 输出每个指标的平均值
print("Average Accuracy:", sum(accuracy_scores) / len(accuracy_scores))
print("Average Precision:", sum(precision_scores) / len(precision_scores))
print("Average Recall:", sum(recall_scores) / len(recall_scores))
print("Average F1 Score:", sum(f1_scores) / len(f1_scores))
if roc_auc_scores:
    print("Average ROC AUC:", sum(roc_auc_scores) / len(roc_auc_scores))
print("Average Specificity:", sum(specificity_scores) / len(specificity_scores))

Average Accuracy: 0.9242708626887912
Average Precision: 0.9303832930839114
Average Recall: 0.924214769030756
Average F1 Score: 0.9239838309189986
Average ROC AUC: 0.9753659786638409
Average Specificity: 0.864859954030004


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

model = RandomForestClassifier()

# 创建 KFold 交叉验证分割器
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 初始化指标列表
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []

# 进行10×10交叉验证
for train_index, test_index in kf.split(X_train):
    # 分割数据，使用 .iloc 来根据行索引选择数据
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # 训练模型
    model.fit(X_train_fold, y_train_fold)
    
    # 进行预测
    y_pred_fold = model.predict(X_test_fold)
    
    # 计算并存储每个指标
    accuracy = accuracy_score(y_test_fold, y_pred_fold)
    accuracy_scores.append(accuracy)
    
    precision = precision_score(y_test_fold, y_pred_fold, average='macro')
    precision_scores.append(precision)
    
    recall = recall_score(y_test_fold, y_pred_fold, average='macro')
    recall_scores.append(recall)
    
    f1 = f1_score(y_test_fold, y_pred_fold, average='macro')
    f1_scores.append(f1)
    
    # 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
    if len(set(y_test_fold)) == 2:
        y_scores_fold = model.predict_proba(X_test_fold)[:, 1]  # 获取正类的概率
        roc_auc = roc_auc_score(y_test_fold, y_scores_fold)
        roc_auc_scores.append(roc_auc)
    
    # 计算Specificity
    conf_mat_fold = confusion_matrix(y_test_fold, y_pred_fold)
    tn, fp, fn, tp = conf_mat_fold.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificity_scores.append(specificity)

# 输出每个指标的平均值
print("Average Accuracy:", sum(accuracy_scores) / len(accuracy_scores))
print("Average Precision:", sum(precision_scores) / len(precision_scores))
print("Average Recall:", sum(recall_scores) / len(recall_scores))
print("Average F1 Score:", sum(f1_scores) / len(f1_scores))
if roc_auc_scores:
    print("Average ROC AUC:", sum(roc_auc_scores) / len(roc_auc_scores))
print("Average Specificity:", sum(specificity_scores) / len(specificity_scores))

Average Accuracy: 0.9351347305764778
Average Precision: 0.9351605796407612
Average Recall: 0.9351279270728108
Average F1 Score: 0.9351191911864405
Average ROC AUC: 0.9858176506361902
Average Specificity: 0.9312483088110477


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 创建随机森林分类器
model = RandomForestClassifier()

# 定义参数网格
param_grid = {
    'n_estimators': [300],
    'max_depth': [None],
    'min_samples_split': [2],
    'min_samples_leaf': [1]
}

# 创建 KFold 交叉验证分割器
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 设置网格搜索，使用 KFold 交叉验证，并指定refit参数
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kf, scoring='accuracy', refit=True, verbose=1)

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best parameters:", grid_search.best_params_)

# 获取最佳模型
best_rf = grid_search.best_estimator_

# 使用最佳模型进行交叉验证预测
y_pred = cross_val_predict(best_rf, X_train, y_train, cv=kf)

# 计算所有指标的平均值
accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred, average='macro')
recall = recall_score(y_train, y_pred, average='macro')
f1 = f1_score(y_train, y_pred, average='macro')

# 如果是二分类问题，计算ROC AUC
if len(set(y_train)) == 2:
    y_scores = best_rf.predict_proba(X_train)[:, 1]
    roc_auc = roc_auc_score(y_train, y_scores)
else:
    roc_auc = None  # 对于多分类问题，ROC AUC需要不同的处理

# 打印分类报告
print(classification_report(y_train, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_train, y_pred)
print(conf_mat)

# 输出每个指标的平均值
print("Average Accuracy:", accuracy)
print("Average Precision:", precision)
print("Average Recall:", recall)
print("Average F1 Score:", f1)
if roc_auc is not None:
    print("Average ROC AUC:", roc_auc)

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
print("Average Specificity:", specificity)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      7721
           1       0.93      0.94      0.94      7742

    accuracy                           0.94     15463
   macro avg       0.94      0.94      0.94     15463
weighted avg       0.94      0.94      0.94     15463

[[7199  522]
 [ 441 7301]]
Average Accuracy: 0.9377223048567548
Average Precision: 0.9377755835424206
Average Recall: 0.9377150759313331
Average F1 Score: 0.9377195948881436
Average ROC AUC: 0.9999999999999999
Average Specificity: 0.9323921771791219


In [11]:
# 获取特征重要性
feature_importances = best_rf.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

       Feature  Importance
56  FHS-SF10_R    0.053630
83    Fitness3    0.049775
52   FHS-SF6_R    0.049533
61  EQ-5D-5L-5    0.043915
88     Stress2    0.033408
..         ...         ...
79     EBS-SF6    0.003855
68  HLS-SF12-7    0.003823
42      NGSES4    0.003802
14       smoke    0.003407
3     chaiqian    0.001653

[96 rows x 2 columns]
       Feature  Importance
56  FHS-SF10_R    0.053630
83    Fitness3    0.049775
52   FHS-SF6_R    0.049533
61  EQ-5D-5L-5    0.043915
88     Stress2    0.033408
55   FHS-SF9_R    0.030141
85    Fitness5    0.025077
87     Stress1    0.022799
49     FHS-SF3    0.022551
51     FHS-SF5    0.020880


In [12]:
importance_df['Cumulative Importance'] = importance_df['Importance'].cumsum()
threshold = 0.8  # 这里设定80%的阈值
N = importance_df[importance_df['Cumulative Importance'] <= threshold].shape[0]

In [13]:
# 打印特征重要性排序
print(importance_df.head(N))

               Feature  Importance  Cumulative Importance
56          FHS-SF10_R    0.053630               0.053630
83            Fitness3    0.049775               0.103405
52           FHS-SF6_R    0.049533               0.152938
61          EQ-5D-5L-5    0.043915               0.196854
88             Stress2    0.033408               0.230261
55           FHS-SF9_R    0.030141               0.260402
85            Fitness5    0.025077               0.285478
87             Stress1    0.022799               0.308278
49             FHS-SF3    0.022551               0.330828
51             FHS-SF5    0.020880               0.351708
23              BFI7_R    0.020026               0.371734
89             Stress3    0.016584               0.388319
50             FHS-SF4    0.016033               0.404352
48             FHS-SF2    0.015674               0.420025
47             FHS-SF1    0.015649               0.435674
12          brosis_Num    0.014402               0.450077
16            

In [16]:
from joblib import dump, load
dump(best_rf, 'best_rf_itemanx.joblib')

['best_rf_itemanx.joblib']

In [14]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 参数网格
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 41, 51]
}

# 创建 LightGBM 分类器
gbm = LGBMClassifier()

# 创建 KFold 交叉验证分割器
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 创建网格搜索对象，使用 KFold 交叉验证
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=kf, scoring='accuracy', verbose=1)

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 最佳参数和最佳模型
print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# 使用最佳模型进行交叉验证预测
y_pred = cross_val_predict(best_model, X_train, y_train, cv=kf)

# 计算所有指标的平均值
accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred, average='macro')
recall = recall_score(y_train, y_pred, average='macro')
f1 = f1_score(y_train, y_pred, average='macro')

# 如果是二分类问题，计算ROC AUC
if len(set(y_train)) == 2:
    y_scores = best_model.predict_proba(X_train)[:, 1]
    roc_auc = roc_auc_score(y_train, y_scores)
else:
    roc_auc = None  # 对于多分类问题，ROC AUC需要不同的处理

# 打印分类报告
print(classification_report(y_train, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_train, y_pred)
print(conf_mat)

# 输出每个指标的平均值
print("Average Accuracy:", accuracy)
print("Average Precision:", precision)
print("Average Recall:", recall)
print("Average F1 Score:", f1)
if roc_auc is not None:
    print("Average ROC AUC:", roc_auc)

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
print("Average Specificity:", specificity)

Fitting 10 folds for each of 45 candidates, totalling 450 fits
[LightGBM] [Info] Number of positive: 6964, number of negative: 6952
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002884 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 810
[LightGBM] [Info] Number of data points in the train set: 13916, number of used features: 96
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500431 -> initscore=0.001725
[LightGBM] [Info] Start training from score 0.001725
[LightGBM] [Info] Number of positive: 6978, number of negative: 6938
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002584 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 810
[LightGBM] [Info] Number of data points in the train set

In [15]:
# 获取特征重要性
feature_importances = best_model.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

          Feature  Importance
16            BMI        1550
6   family_income         644
8             age         617
9             edu         548
87        Stress1         491
..            ...         ...
69     HLS-SF12-8         109
43         NGSES5         109
59     EQ-5D-5L-3          96
58     EQ-5D-5L-2          94
3        chaiqian          93

[96 rows x 2 columns]
          Feature  Importance
16            BMI        1550
6   family_income         644
8             age         617
9             edu         548
87        Stress1         491
88        Stress2         436
61     EQ-5D-5L-5         432
17         BFI1_R         414
89        Stress3         405
4     home_mianji         398


In [17]:
from joblib import dump, load
dump(best_model, 'best_gbm_itemanx.joblib')

['best_gbm_itemanx.joblib']

In [18]:
from joblib import load
best_rf = load('best_rf_itemanx.joblib')
best_gbm = load('best_gbm_itemanx.joblib')

# 已经训练好随机森林模型
feature_importances_rf = best_rf.feature_importances_
feature_names = X.columns

# 将特征重要性和特征名组合成DataFrame
importance_df_rf = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances_rf})

feature_importances_lgbm = best_gbm.feature_importances_

# 将特征重要性和特征名组合成DataFrame
importance_df_lgbm = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances_lgbm})

In [19]:
# 找到随机森林模型特征重要性中的最大值
max_importance_rf = importance_df_rf['Importance'].max()

# 对随机森林模型的特征重要性进行标准化，使最大特征值为100
importance_df_rf['Standardized Importance'] = importance_df_rf['Importance'] * (100 / max_importance_rf)

# 找到LightGBM模型特征重要性中的最大值
max_importance_lgbm = importance_df_lgbm['Importance'].max()

# 对LightGBM模型的特征重要性进行标准化，使最大特征值为100
importance_df_lgbm['Standardized Importance'] = importance_df_lgbm['Importance'] * (100 / max_importance_lgbm)


In [20]:
# 按照特征名进行合并
merged_importance_df = pd.merge(importance_df_rf, importance_df_lgbm, on='Feature')

# 计算每个特征的重要性平均值
merged_importance_df['Average Importance'] = (merged_importance_df['Standardized Importance_x'] + merged_importance_df['Standardized Importance_y']) / 2

print(merged_importance_df)

               Feature  Importance_x  Standardized Importance_x  Importance_y  \
0           liveStrict      0.005865                  10.936710           193   
1                hukou      0.005797                  10.809320           199   
2              homeNum      0.006387                  11.909115           317   
3             chaiqian      0.001653                   3.083101            93   
4          home_mianji      0.007276                  13.566742           398   
..                 ...           ...                        ...           ...   
91  Emontion_regulate2      0.011837                  22.071944           352   
92  Emontion_regulate3      0.006858                  12.788402           333   
93  Emontion_regulate4      0.008589                  16.014763           333   
94  Emontion_regulate5      0.007645                  14.255865           290   
95  Emontion_regulate6      0.007065                  13.172965           370   

    Standardized Importance

In [21]:
pd.options.display.max_rows = None
print(merged_importance_df)

               Feature  Importance_x  Standardized Importance_x  Importance_y  \
0           liveStrict      0.005865                  10.936710           193   
1                hukou      0.005797                  10.809320           199   
2              homeNum      0.006387                  11.909115           317   
3             chaiqian      0.001653                   3.083101            93   
4          home_mianji      0.007276                  13.566742           398   
5               zhaiwu      0.004117                   7.676964           208   
6        family_income      0.007670                  14.301697           644   
7               gender      0.008603                  16.040522           233   
8                  age      0.010721                  19.989677           617   
9                  edu      0.008810                  16.428010           548   
10            marriage      0.012181                  22.711980           191   
11           child_Num      

In [22]:
df = pd.DataFrame(merged_importance_df)
df.to_excel(r'C:\Users\Lenovo\Desktop\ML_health\merged_importance_df_itemanx.xlsx', index=False, engine='openpyxl')

In [24]:
import shap
# 假设 X 是特征数据，model 是你的训练好的模型
explainer = shap.TreeExplainer(best_rf)
shap_values = explainer.shap_values(X)